In [6]:
def read_pixels(file_name: str) -> list[tuple[int]]:
    with open(file_name, "r") as f:
        line = f.readline().strip()
        words = line.split(" ")
        pixels = []
        for i in range(0, len(words), 3):
            pixels.append((int(words[i]), int(words[i + 1]), int(words[i + 2])))
        return pixels

In [7]:
# テキストファイルを改行した結果と照らし合わせる
pixels = read_pixels("./data/image1.txt")
with open("out/pixels", mode="w") as f:
    f.write(str(pixels))
assert len(pixels) == 388800 / 3

## (2)


In [8]:
# とりあえず実験として、直前の(255, 255, 255)から次の(255, 255, 255)の間隔がどうなっているかを見る。
pixels = read_pixels("./data/image1.txt")

rows = []
width = 0
for i, pixel in enumerate(pixels):
    if pixel[0] == 255 and pixel[1] == 255 and pixel[2] == 255:
        rows.append({"width": width, "to": i})
        width = 0
    else:
        width += 1

print(rows)

[{'width': 479, 'to': 479}, {'width': 479, 'to': 959}, {'width': 479, 'to': 1439}, {'width': 479, 'to': 1919}, {'width': 479, 'to': 2399}, {'width': 479, 'to': 2879}, {'width': 479, 'to': 3359}, {'width': 479, 'to': 3839}, {'width': 479, 'to': 4319}, {'width': 479, 'to': 4799}, {'width': 479, 'to': 5279}, {'width': 479, 'to': 5759}, {'width': 479, 'to': 6239}, {'width': 479, 'to': 6719}, {'width': 479, 'to': 7199}, {'width': 479, 'to': 7679}, {'width': 479, 'to': 8159}, {'width': 479, 'to': 8639}, {'width': 479, 'to': 9119}, {'width': 479, 'to': 9599}, {'width': 479, 'to': 10079}, {'width': 479, 'to': 10559}, {'width': 479, 'to': 11039}, {'width': 479, 'to': 11519}, {'width': 479, 'to': 11999}, {'width': 479, 'to': 12479}, {'width': 479, 'to': 12959}, {'width': 479, 'to': 13439}, {'width': 479, 'to': 13919}, {'width': 479, 'to': 14399}, {'width': 479, 'to': 14879}, {'width': 479, 'to': 15359}, {'width': 479, 'to': 15839}, {'width': 479, 'to': 16319}, {'width': 479, 'to': 16799}, {'widt

In [9]:
# おそらく479ピクセルだが、念のためそれ以上の値がないか確かめる
max = 479
for row in rows:
    if row["width"] > max:
        max = row["width"]
print(max)

479


## (3)


In [10]:
# とりあえず明るさを配列に加える
pixels = read_pixels("./data/image1.txt")

brightnesses = []
for i, pixel in enumerate(pixels):
    brightnesses.append((i, pixel[0] ** 2 + pixel[1] ** 2 + pixel[2] ** 2))

# ファイルに書き出す。これは検証でも使える。
with open("out/brightnesses", mode="w") as f:
    f.write("\n".join([str(brightness) for brightness in brightnesses]))

In [11]:
255**2 * 3 == 195075

True

In [12]:
# ソートして中央の値を取る
sorted_brightnesses = sorted(brightnesses, key=lambda x: x[1])
print(sorted_brightnesses[len(sorted_brightnesses) // 2])

# 目視でも確認する。
with open("out/sorted_brightnesses", mode="w") as f:
    f.write("\n".join([str(brightness) for brightness in sorted_brightnesses]))

(11997, 79896)


## (4)


In [13]:
def append_index_and_brightness(pixels: list[tuple[int]]) -> list[tuple[int]]:
    with_index_and_brightness = []
    for i, pixel in enumerate(pixels):
        with_index_and_brightness.append(
            (
                i,
                pixel[0],
                pixel[1],
                pixel[2],
                pixel[0] ** 2 + pixel[1] ** 2 + pixel[2] ** 2,
            )
        )

    return sorted(with_index_and_brightness, key=lambda x: x[4])

In [14]:
origin = [(1, 1, 1), (0, 0, 0)]
expected = [(1, 0, 0, 0, 0), (0, 1, 1, 1, 3)]
actual = append_index_and_brightness(origin)
print(actual)
assert expected == actual

[(1, 0, 0, 0, 0), (0, 1, 1, 1, 3)]


In [15]:
# とりあえず整形したデータを眺める
pixels2 = read_pixels("./data/image2.txt")
with open("out/pixels2", mode="w") as f:
    f.write("\n".join([str(pixel2) for pixel2 in pixels2]))

sorted_brightnesses2 = append_index_and_brightness(pixels2)
print(len(sorted_brightnesses2))

1640000


In [16]:
def calc_initial_cluster_means(pixels, k):
    means = []
    n = len(pixels)
    for i in range(k):
        means.append({"core": pixels[n * i // k], "members": []})
    return means

In [17]:
k = 4
actual = calc_initial_cluster_means(sorted_brightnesses2, k)
print(actual)

[{'core': (247908, 0, 0, 0, 0), 'members': []}, {'core': (1399112, 88, 88, 88, 23232), 'members': []}, {'core': (1261515, 124, 124, 124, 46128), 'members': []}, {'core': (677684, 155, 155, 155, 72075), 'members': []}]


## (5) k 近傍法


In [18]:
# 複雑なのでテストがあった方が良い。とりあえず、2パターン考えよう。
# ケース1: 黒ピクセル500, 白ピクセル500の画像。k=100とすると、i=25ではまず黒, i=75ではまず白になるはずだ
# → これは誤りだった（何も分類されないクラスターができてバグの原因になる）
# ケース2: ランダムな0~255の値を取る白黒画像。十分なピクセル数があればi=k/2で(128, 128, 128)くらいになるはずだ。

In [19]:
def testcase1():
    pixels = []
    for _ in range(0, 500):
        pixels.append((0, 0, 0))
        pixels.append((255, 255, 255))
    return pixels


testcase1_pixels = testcase1()
with open("out/testcase1_pixels", mode="w") as f:
    f.write("\n".join([str(pixel) for pixel in testcase1_pixels]))

In [20]:
import random


def testcase2():
    pixels = []
    for _ in range(0, 100000):
        brightness = random.randint(0, 255)
        pixels.append((brightness, brightness, brightness))
    return pixels


testcase2_pixels = testcase2()
with open("out/testcase2_pixels", mode="w") as f:
    f.write("\n".join([str(pixel) for pixel in testcase2_pixels]))

In [21]:
def calc_distance(pix1, pix2) -> int:
    _, r1, g1, b1, _ = pix1
    _, r2, g2, b2, _ = pix2
    return abs(r1 - r2) + abs(g1 - g2) + abs(b1 - b2)

In [22]:
expected = 30
actual = calc_distance(
    (0, 100, 100, 100, 300000), (1, 100, 90, 120, 100**2 + 90**2 + 120**2)
)
assert actual == expected

In [23]:
def members_updated(clusters, members):
    MAX_INT = 2**63 - 1

    assigned = [{"core": cluster["core"], "members": []} for cluster in clusters]
    for pixel in members:
        champion = -1
        champion_distance = MAX_INT
        for index, cluster in enumerate(assigned):
            distance = calc_distance(pixel, cluster["core"])
            if distance <= champion_distance:
                champion = index
                champion_distance = distance

        assigned[champion]["members"].append(pixel)
    return assigned

In [24]:
original_members = [
    (0, 0, 0, 0, 0),
    (0, 1, 1, 1, 3),
    (0, 100, 100, 100, 30000),
    (0, 100, 100, 100, 30000),
]
original_clusters = [
    {
        "core": (0, 1, 1, 1, 3),
        "members": [(0, 100, 100, 100, 30000), (0, 100, 100, 100, 30000)],
    },
    {"core": (0, 100, 100, 100, 30000), "members": [(0, 0, 0, 0, 0), (0, 1, 1, 1, 3)]},
]
expected = clusters = [
    {"core": (0, 1, 1, 1, 3), "members": [(0, 0, 0, 0, 0), (0, 1, 1, 1, 3)]},
    {
        "core": (0, 100, 100, 100, 30000),
        "members": [(0, 100, 100, 100, 30000), (0, 100, 100, 100, 30000)],
    },
]
actual = members_updated(original_clusters, original_members)
print(actual)
assert actual == expected

[{'core': (0, 1, 1, 1, 3), 'members': [(0, 0, 0, 0, 0), (0, 1, 1, 1, 3)]}, {'core': (0, 100, 100, 100, 30000), 'members': [(0, 100, 100, 100, 30000), (0, 100, 100, 100, 30000)]}]


In [25]:
def means_updated(cluster):
    MAX_INT = 2**63 - 1
    members = cluster["members"]
    r_total, g_total, b_total = -1, -1, -1
    for member in members:
        _, r, g, b, _ = member
        r_total += r
        g_total += g
        b_total += b
    r_c, g_c, b_c = (
        r_total / len(members),
        g_total / len(members),
        b_total / len(members),
    )

    champion = None
    champion_dist = MAX_INT
    for member in members:
        dist = calc_distance((-1, r_c, g_c, b_c, -1), member)
        # print(f"{champion=}, {champion_dist=}, {member=}, {dist=}")
        if dist <= champion_dist:
            champion = member
            champion_dist = dist

    return {"core": champion, "members": members}

In [26]:
origin = {
    "core": (0, 0, 0, 0, 0),
    "members": [(0, 0, 0, 0, 0), (0, 100, 100, 100, 30000), (0, 200, 200, 200, 120000)],
}
expected = {
    "core": (0, 100, 100, 100, 30000),
    "members": [(0, 0, 0, 0, 0), (0, 100, 100, 100, 30000), (0, 200, 200, 200, 120000)],
}
actual = means_updated(origin)

assert actual == expected

In [27]:
from datetime import datetime


def k_means(pixels: list[tuple[int]], k: int, iteration: int):
    with_index_and_brightness = append_index_and_brightness(pixels)
    clusters = calc_initial_cluster_means(with_index_and_brightness, k)
    # print(clusters)

    for i in range(iteration):
        print(f"{datetime.now().isoformat()}, {i=}, start")
        member_updated_clusters = members_updated(clusters, with_index_and_brightness)
        print(f"{datetime.now().isoformat()}, {i=}, members updated")
        clusters = [means_updated(cluster) for cluster in member_updated_clusters]
        print(f"{datetime.now().isoformat()}, {i=}, means updated")

    return clusters

In [28]:
k_means_clusters = k_means(testcase2_pixels, 128, 3)
with open("out/k_means", mode="w") as f:
    f.write("\n".join([str(cluster) for cluster in k_means_clusters]))

2024-06-13T15:51:16.506632, i=0
2024-06-13T15:51:17.888451, i=0, member updated
2024-06-13T15:51:17.908498, i=0, means updated
2024-06-13T15:51:17.908523, i=1
2024-06-13T15:51:19.264138, i=1, member updated
2024-06-13T15:51:19.286468, i=1, means updated
2024-06-13T15:51:19.286496, i=2
2024-06-13T15:51:20.639808, i=2, member updated
2024-06-13T15:51:20.661535, i=2, means updated


In [30]:
pixels2 = read_pixels("./data/image2.txt")
sorted_brightnesses2 = append_index_and_brightness(pixels2)
k_neighbor_clusters2 = k_means(sorted_brightnesses2, 128, 10)
with open("out/k_neighbor_clusters2", mode="w") as f:
    f.write("\n".join([str(cluster) for cluster in k_neighbor_clusters2]))

2024-06-13T15:56:26.881984, i=0
2024-06-13T15:56:52.689860, i=0, member updated
2024-06-13T15:56:53.025382, i=0, means updated
2024-06-13T15:56:53.025466, i=1
2024-06-13T15:57:18.651179, i=1, member updated
2024-06-13T15:57:19.021414, i=1, means updated
2024-06-13T15:57:19.021537, i=2
2024-06-13T15:57:45.071684, i=2, member updated
2024-06-13T15:57:45.379255, i=2, means updated
2024-06-13T15:57:45.379345, i=3
2024-06-13T15:58:10.717108, i=3, member updated
2024-06-13T15:58:11.034771, i=3, means updated
2024-06-13T15:58:11.034869, i=4
2024-06-13T15:58:36.253100, i=4, member updated
2024-06-13T15:58:36.561925, i=4, means updated
2024-06-13T15:58:36.562009, i=5
2024-06-13T15:59:01.158211, i=5, member updated
2024-06-13T15:59:01.463501, i=5, means updated
2024-06-13T15:59:01.463601, i=6
2024-06-13T15:59:26.786871, i=6, member updated
2024-06-13T15:59:27.091018, i=6, means updated
2024-06-13T15:59:27.091104, i=7
2024-06-13T15:59:52.056068, i=7, member updated
2024-06-13T15:59:52.359110, i=7